<a href="https://colab.research.google.com/github/Keita-S593/rindoku/blob/main/AutoGluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考: https://aws.amazon.com/jp/builders-flash/202201/autogluon-tabular-tutorials/?awsf.filter-name=*all 

In [1]:
!pip install autogluon==0.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 3.8 MB/s 
     |████████████████████████████████| 52 kB 1.1 MB/s 
     |████████████████████████████████| 273 kB 37.8 MB/s 
     |████████████████████████████████| 352 kB 48.2 MB/s 
     |████████████████████████████████| 212 kB 42.6 MB/s 
     |████████████████████████████████| 22.3 MB 1.6 MB/s 
     |████████████████████████████████| 4.2 MB 36.4 MB/s 
     |████████████████████████████████| 27.4 MB 64.8 MB/s 
     |████████████████████████████████| 132 kB 57.2 MB/s 
     |████████████████████████████████| 1.3 MB 35.2 MB/s 
     |████████████████████████████████| 119 kB 55.6 MB/s 
     |████████████████████████████████| 3.0 MB 50.4 MB/s 
     |████████████████████████████████| 296 kB 52.7 MB/s 
     |████████████████████████████████| 67.3 MB 14 kB/s 
     |████████████████████████████████| 166.7 MB 16 kB/s 
     |█████████████████████████████

# インポート

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.metrics import confusion_matrix, roc_auc_score

# 学習データ読み込み

In [3]:
df = pd.read_csv('/content/drive/MyDrive/輪読/titanic/train.csv')

In [7]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 前処理

In [5]:
drop_col = ["PassengerId", "Name", "Ticket", "Cabin"]
train_data = df.drop(drop_col, axis=1)

In [8]:
train_data.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S


# とりあえずデフォルトの設定で学習

In [ ]:
dir_default = '/content/drive/MyDrive/輪読/model/agModels_default'
label_column = 'Survived'

In [ ]:
predictor = TabularPredictor(label=label_column, path=dir_default).fit(train_data=train_data)

INFO:autogluon.tabular.learner.default_learner:Beginning AutoGluon training ...
INFO:autogluon.tabular.learner.default_learner:AutoGluon will save models to "/content/drive/MyDrive/輪読/model/agModels_default/"
INFO:autogluon.tabular.learner.default_learner:AutoGluon Version:  0.3.1
INFO:autogluon.tabular.learner.default_learner:Train Data Rows:    891
INFO:autogluon.tabular.learner.default_learner:Train Data Columns: 7
INFO:autogluon.tabular.learner.default_learner:Preprocessing data ...
Level 25:autogluon.core.utils.utils:AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
INFO:autogluon.core.utils.utils:	2 unique label values:  [0, 1]
Level 25:autogluon.core.utils.utils:	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
INFO:autogluon.core.data.label_cleaner:Selected class <--> label mapping:  class 1 

## 学習の詳細

In [ ]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.854749       0.039632  1.714237                0.000991           0.465882            2       True         13
1         LightGBMLarge   0.843575       0.016522  0.640542                0.016522           0.640542            1       True         12
2               XGBoost   0.832402       0.010714  0.220638                0.010714           0.220638            1       True         11
3              CatBoost   0.826816       0.005918  0.752617                0.005918           0.752617            1       True          7
4              LightGBM   0.821229       0.012486  0.430301                0.012486           0.430301            1       True          4
5       NeuralNetFastAI   0.810056       0.028103  1.612583                0.028103           1.612583        

# 特徴量の重要度

今回は全て学習に使ってしまったのでサンプリングで代用

In [ ]:
sampled_data = train_data.sample(frac=0.3, random_state=42)

In [ ]:
feature_importances = predictor.feature_importance(sampled_data)

Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_best/models/WeightedEnsemble_L3/model.pkl
INFO:autogluon.core.utils.utils:Computing feature importance via permutation shuffling for 7 features using 267 rows with 3 shuffle sets...
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_best/models/CatBoost_BAG_L1/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_best/models/ExtraTreesEntr_BAG_L1/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_best/models/ExtraTreesGini_BAG_L1/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_best/models/KNeighborsDist_BAG_L1/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_best/models/KNeighborsUnif_BAG_L1/model.pkl
Level 15:autogluon.cor

In [ ]:
feature_importances

,importance,stddev,p_value,n,p99_high,p99_low
Sex,0.212235,0.042594,0.006581,3,0.456301,-0.031831
Pclass,0.122347,0.021624,0.005126,3,0.246253,-0.001559
Age,0.108614,0.012974,0.002361,3,0.182958,0.034271
Fare,0.088639,0.002162,0.000099,3,0.101030,0.076249
SibSp,0.024969,0.005721,0.008527,3,0.057751,-0.007814
Parch,0.023720,0.002162,0.001379,3,0.036111,0.011330
Embarked,0.023720,0.009426,0.024405,3,0.077730,-0.030289


## 提出

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/輪読/titanic/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/輪読/titanic/gender_submission.csv')

In [ ]:
X_sub = df_sub.drop(drop_col, axis=1)

In [ ]:
y_pred_sub = predictor.predict(X_sub)

In [ ]:
sub['Survived'] = y_pred_sub
sub

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
sub.to_csv('/content/drive/MyDrive/輪読/submit/submit_agModel_default.csv', index=False)

LBで0.7799

# 設定を変更

デフォルトではmetricとしてaccuracyが用いられており、presetsはmedium_quality_faster_trainで性能とコストを両立する様になっている。これらを変更してより良いモデルを作ってみる

## metricをaucにする

In [ ]:
dir_auc = '/content/drive/MyDrive/輪読/model/agModels_auc'
eval_metric = "roc_auc"
label_column = 'Survived'

In [ ]:
#verbosityを設定するとモデルの詳細を出力
predictor = TabularPredictor(label=label_column, path=dir_auc, eval_metric=eval_metric, verbosity=3).fit(train_data=train_data)

INFO:root:============ fit kwarg info ============
INFO:root:User Specified kwargs:
INFO:root:{}
INFO:root:Full kwargs:
INFO:root:{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': False,
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'quantile_levels': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 3}
INFO:root:========================================
INFO:autogluon.tabular.learner.default_learner:Beginning AutoGluon training ...
INFO:autogluon.tabular.learner.default_learner:AutoGluon will save models to "/content/drive/MyDrive/輪読/model/agModels_auc_best/"
INFO:autogluon.tabular.learner.default_learner:AutoGluon Version

[50]	train_set's binary_logloss: 0.417901	valid_set's binary_logloss: 0.477585
[100]	train_set's binary_logloss: 0.37708	valid_set's binary_logloss: 0.467542
[150]	train_set's binary_logloss: 0.351245	valid_set's binary_logloss: 0.470153
[200]	train_set's binary_logloss: 0.334496	valid_set's binary_logloss: 0.474263
[250]	train_set's binary_logloss: 0.322369	valid_set's binary_logloss: 0.47991


INFO:autogluon.tabular.trainer.abstract_trainer:	0.01s	 = Validation runtime
Level 15:autogluon.core.utils.savers.save_pkl:Saving /content/drive/MyDrive/輪読/model/agModels_auc_best/models/trainer.pkl
INFO:autogluon.tabular.trainer.abstract_trainer:Fitting model: LightGBM ...
Level 15:autogluon.core.models.abstract.abstract_model:	Fitting LightGBM with 'num_gpus': 0, 'num_cpus': 2
Level 15:autogluon.tabular.models.lgb.lgb_model:Training Gradient Boosting Model for 10000 rounds...
Level 15:autogluon.tabular.models.lgb.lgb_model:with the following hyperparameter settings:
Level 15:autogluon.tabular.models.lgb.lgb_model:{'num_threads': -1, 'learning_rate': 0.05, 'objective': 'binary', 'verbose': -1, 'boosting_type': 'gbdt', 'two_round': True}
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'v

[50]	train_set's binary_logloss: 0.306473	valid_set's binary_logloss: 0.472978
[100]	train_set's binary_logloss: 0.222475	valid_set's binary_logloss: 0.481468
[150]	train_set's binary_logloss: 0.17402	valid_set's binary_logloss: 0.520898


INFO:autogluon.tabular.trainer.abstract_trainer:	0.8291	 = Validation score   (roc_auc)
INFO:autogluon.tabular.trainer.abstract_trainer:	0.5s	 = Training   runtime
INFO:autogluon.tabular.trainer.abstract_trainer:	0.01s	 = Validation runtime
Level 15:autogluon.core.utils.savers.save_pkl:Saving /content/drive/MyDrive/輪読/model/agModels_auc_best/models/trainer.pkl
INFO:autogluon.tabular.trainer.abstract_trainer:Fitting model: RandomForestGini ...
Level 15:autogluon.core.models.abstract.abstract_model:	Fitting RandomForestGini with 'num_gpus': 0, 'num_cpus': 2
Level 15:autogluon.core.utils.savers.save_pkl:Saving /content/drive/MyDrive/輪読/model/agModels_auc_best/models/RandomForestGini/model.pkl
INFO:autogluon.tabular.trainer.abstract_trainer:	0.8175	 = Validation score   (roc_auc)
INFO:autogluon.tabular.trainer.abstract_trainer:	1.14s	 = Training   runtime
INFO:autogluon.tabular.trainer.abstract_trainer:	0.21s	 = Validation runtime
Level 15:autogluon.core.utils.savers.save_pkl:Saving /conte

...

Level 15:autogluon.core.utils.savers.save_pkl:Saving /content/drive/MyDrive/輪読/model/agModels_auc_best/models/XGBoost/model.pkl
INFO:autogluon.tabular.trainer.abstract_trainer:	0.8403	 = Validation score   (roc_auc)
INFO:autogluon.tabular.trainer.abstract_trainer:	0.37s	 = Training   runtime
INFO:autogluon.tabular.trainer.abstract_trainer:	0.01s	 = Validation runtime
Level 15:autogluon.core.utils.savers.save_pkl:Saving /content/drive/MyDrive/輪読/model/agModels_auc_best/models/trainer.pkl
INFO:autogluon.tabular.trainer.abstract_trainer:Fitting model: NeuralNetMXNet ...
Level 15:autogluon.core.models.abstract.abstract_model:	Fitting NeuralNetMXNet with 'num_gpus': 0, 'num_cpus': 2
ERROR:autogluon.tabular.trainer.abstract_trainer:	Warning: Exception caused NeuralNetMXNet to fail during training (ImportError)... Skipping this model.
ERROR:autogluon.tabular.trainer.abstract_trainer:		Unable to import dependency mxnet. A quick tip is to install via `pip install mxnet --upgrade`, or `pip insta

[50]	train_set's binary_logloss: 0.262627	valid_set's binary_logloss: 0.466871
[100]	train_set's binary_logloss: 0.143286	valid_set's binary_logloss: 0.501164
[150]	train_set's binary_logloss: 0.0863635	valid_set's binary_logloss: 0.56756


Level 15:autogluon.core.utils.savers.save_pkl:Saving /content/drive/MyDrive/輪読/model/agModels_auc_best/models/LightGBMLarge/model.pkl
INFO:autogluon.tabular.trainer.abstract_trainer:	0.8196	 = Validation score   (roc_auc)
INFO:autogluon.tabular.trainer.abstract_trainer:	0.79s	 = Training   runtime
INFO:autogluon.tabular.trainer.abstract_trainer:	0.01s	 = Validation runtime
Level 15:autogluon.core.utils.savers.save_pkl:Saving /content/drive/MyDrive/輪読/model/agModels_auc_best/models/trainer.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_auc_best/models/CatBoost/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_auc_best/models/ExtraTreesEntr/model.pkl


[200]	train_set's binary_logloss: 0.0598874	valid_set's binary_logloss: 0.643024


Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_auc_best/models/ExtraTreesGini/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_auc_best/models/KNeighborsDist/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_auc_best/models/KNeighborsUnif/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_auc_best/models/LightGBM/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_auc_best/models/LightGBMLarge/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_auc_best/models/LightGBMXT/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_auc_best/models/NeuralNetFastAI/model.pkl
Level 15:autogluon.core.utils.loaders.load_p

### 提出

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/輪読/titanic/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/輪読/titanic/gender_submission.csv')

In [ ]:
X_sub = df_sub.drop(drop_col, axis=1)

In [ ]:
y_pred_sub = predictor.predict(X_sub)

Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_auc_best/models/WeightedEnsemble_L2/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_auc_best/models/CatBoost/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_auc_best/models/ExtraTreesGini/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_auc_best/models/KNeighborsDist/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_auc_best/models/LightGBM/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_auc_best/models/LightGBMLarge/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_auc_best/models/LightGBMXT/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl

In [ ]:
sub['Survived'] = y_pred_sub
sub

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
sub.to_csv('/content/drive/MyDrive/輪読/submit/submit_agModel_auc_best.csv', index=False)

## presetsをbest_qualityにする

In [ ]:
dir_best = '/content/drive/MyDrive/輪読/model/agModels_best'
label_column = 'Survived'

# 制限時間を設定する。
time_limit = 300 

In [ ]:
predictor = TabularPredictor(label='Survived', path=dir_best).fit(train_data=train_data, presets='best_quality', time_limit=300)

In [ ]:
#verbosityを設定するとモデルの詳細を出力
predictor = TabularPredictor(label=label_column, path=dir_best, verbosity=3).fit(train_data=train_data, presets='best_quality', time_limit=time_limit)

INFO:root:Presets specified: ['best_quality']
INFO:root:============ fit kwarg info ============
INFO:root:User Specified kwargs:
INFO:root:{'auto_stack': True}
INFO:root:Full kwargs:
INFO:root:{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'quantile_levels': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 3}
INFO:root:========================================
INFO:autogluon.tabular.learner.default_learner:Beginning AutoGluon training ... Time limit = 300s
INFO:autogluon.tabular.learner.default_learner:AutoGluon will save models to "/content/drive/MyDrive/輪読/model/agM

[50]	train_set's binary_error: 0.182285	valid_set's binary_error: 0.1875
[100]	train_set's binary_error: 0.154044	valid_set's binary_error: 0.1875
[150]	train_set's binary_error: 0.145058	valid_set's binary_error: 0.1875
[200]	train_set's binary_error: 0.139923	valid_set's binary_error: 0.205357


Level 15:autogluon.core.models.abstract.abstract_model:	Fitting S1F2 with 'num_gpus': 0, 'num_cpus': 2
Level 15:autogluon.tabular.models.lgb.lgb_model:Training Gradient Boosting Model for 10000 rounds...
Level 15:autogluon.tabular.models.lgb.lgb_model:with the following hyperparameter settings:
Level 15:autogluon.tabular.models.lgb.lgb_model:{'num_threads': -1, 'learning_rate': 0.05, 'objective': 'binary', 'verbose': -1, 'boosting_type': 'gbdt', 'two_round': True, 'extra_trees': True}
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
Level 15:autogluon.core.models.abstract.abstract_model:	Fitting S1F3 with 'num_gpus': 0, 'num_cpus': 2


[50]	train_set's binary_error: 0.17715	valid_set's binary_error: 0.223214
[100]	train_set's binary_error: 0.160462	valid_set's binary_error: 0.232143
[150]	train_set's binary_error: 0.146341	valid_set's binary_error: 0.241071


Level 15:autogluon.tabular.models.lgb.lgb_model:Training Gradient Boosting Model for 10000 rounds...
Level 15:autogluon.tabular.models.lgb.lgb_model:with the following hyperparameter settings:
Level 15:autogluon.tabular.models.lgb.lgb_model:{'num_threads': -1, 'learning_rate': 0.05, 'objective': 'binary', 'verbose': -1, 'boosting_type': 'gbdt', 'two_round': True, 'extra_trees': True}
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
Level 15:autogluon.core.models.abstract.abstract_model:	Fitting S1F4 with 'num_gpus': 0, 'num_cpus': 2
Level 15:autogluon.tabular.models.lgb.lgb_model:Training Gradient Boosting Model for 10000 rounds...


...

Level 15:autogluon.core.models.abstract.abstract_model:	Fitting S1F8 with 'num_gpus': 0, 'num_cpus': 2


[150]	train_set's binary_error: 0	valid_set's binary_error: 0.126126


Level 15:autogluon.tabular.models.lgb.lgb_model:Training Gradient Boosting Model for 10000 rounds...
Level 15:autogluon.tabular.models.lgb.lgb_model:with the following hyperparameter settings:
Level 15:autogluon.tabular.models.lgb.lgb_model:{'num_threads': -1, 'learning_rate': 0.03, 'objective': 'binary', 'verbose': -1, 'boosting_type': 'gbdt', 'two_round': True, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 5}
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[50]	train_set's binary_error: 0.00128205	valid_set's binary_error: 0.189189
[100]	train_set's binary_error: 0	valid_set's binary_error: 0.171171
[150]	train_set's binary_error: 0	valid_set's binary_error: 0.18018


Level 15:autogluon.core.utils.savers.save_pkl:Saving /content/drive/MyDrive/輪読/model/agModels_best/models/LightGBMLarge_BAG_L2/utils/oof.pkl
Level 15:autogluon.core.utils.savers.save_pkl:Saving /content/drive/MyDrive/輪読/model/agModels_best/models/LightGBMLarge_BAG_L2/model.pkl
INFO:autogluon.tabular.trainer.abstract_trainer:	0.8608	 = Validation score   (accuracy)
INFO:autogluon.tabular.trainer.abstract_trainer:	11.07s	 = Training   runtime
INFO:autogluon.tabular.trainer.abstract_trainer:	0.06s	 = Validation runtime
Level 15:autogluon.core.utils.savers.save_pkl:Saving /content/drive/MyDrive/輪読/model/agModels_best/models/trainer.pkl
Level 15:autogluon.tabular.trainer.abstract_trainer:Not enough time left to finish repeated k-fold bagging, stopping early ...
INFO:autogluon.tabular.trainer.abstract_trainer:Completed 1/20 k-fold bagging repeats ...
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_best/models/LightGBMXT_BAG_L2/utils/oof.pkl
Le

### 提出

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/輪読/titanic/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/輪読/titanic/gender_submission.csv')

In [ ]:
X_sub = df_sub.drop(drop_col, axis=1)

In [ ]:
y_pred_sub = predictor.predict(X_sub)

Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_best/models/WeightedEnsemble_L3/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_best/models/CatBoost_BAG_L1/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_best/models/ExtraTreesEntr_BAG_L1/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_best/models/ExtraTreesGini_BAG_L1/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_best/models/KNeighborsDist_BAG_L1/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_best/models/KNeighborsUnif_BAG_L1/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: /content/drive/MyDrive/輪読/model/agModels_best/models/LightGBMLarge_BAG_L1/model.pkl
Level 15:autogluon.cor

In [ ]:
sub['Survived'] = y_pred_sub
sub

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
sub.to_csv('/content/drive/MyDrive/輪読/submit/submit_agModel_best.csv', index=False)

LBでで0.77511